# Train Custom Object Detection Model

**Warning:** This notebook assumes that you have run the Object Detection Workspace Setup notebook and have a configured workspace

### Resources

https://github.com/nicknochnack/TFODCourse <br>
**Highly Recommend Watching the Tensorflow Object Detection Walkthrough Video**

### Required Dependicies 
pip install typeguard pytz gin-config tensorflow-addons 

## 1. Import Libraries

In [ ]:
import os
import wget

## 2. Setup Paths to Workspace & Specify Pre-Trained Model

Tensorflow 2 Detection Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

To copy a new model link go to the page above and right click any of the model names in the table, then select copy link address.

In [ ]:
#Set name for your custom object detection model
CUSTOM_MODEL_NAME = 'my_ssd_modnet' 

#Copy link from Tensorflow 2 Destection Model Zoo 
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
#Set to the name of the file downloaded by the link above - DO NOT INCLUDE the file extension
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'

#Sets Label Category - Determined by training dataset
LABEL_MAP_NAME = 'custom_label_map.pbtxt'

TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

#Set project location and directory name
PROJECT_NAME = "C:\\Users\Richard\\Desktop\\ObjDetection_Workspace\\"

paths = {
    'WORKSPACE_PATH': os.path.join(PROJECT_NAME, 'workspace'),
    'SCRIPTS_PATH': os.path.join(PROJECT_NAME,'scripts'),
    'APIMODEL_PATH': os.path.join(PROJECT_NAME,'models'),
    'ANNOTATION_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models', CUSTOM_MODEL_NAME, 'annotations'),
    'IMAGE_PATH': os.path.join(PROJECT_NAME, 'workspace','images'),
    'MODEL_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models'),
    'PRETRAINED_MODEL_PATH': os.path.join(PROJECT_NAME, 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',CUSTOM_MODEL_NAME),
    'LABEL_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',CUSTOM_MODEL_NAME, 'label_map'),
    'OUTPUT_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',CUSTOM_MODEL_NAME, 'final_model'), 
    'TFJS_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH': os.path.join(PROJECT_NAME,'protoc'),
    'TRAIN_PATH': os.path.join(PROJECT_NAME, 'workspace','images','train'),
    'TEST_PATH': os.path.join(PROJECT_NAME, 'workspace','images','test')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(PROJECT_NAME, 'workspace','custom_models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['LABEL_PATH'], LABEL_MAP_NAME)
}

## 3. Create Folders in Workspace

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## 4. Copy Training & Testing Data 

**Training and Testing data must be downloaded or manually created using a program like LabelImg**

**1.** Copy all training images along with thier annotation xml file to workspace/images/train
<br> <br>
**2.** Copy all testing images along with thier annotation xml file to workspace/images/test

## 5. Download Pre-trained Model
**Checks to see if it exist already before downloading**

**Download Location:** workspace/pretrained-models/

In [ ]:
if not os.path.exists(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME)):
    if os.name =='posix':
        !wget {PRETRAINED_MODEL_URL}
        !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    if os.name == 'nt':
        wget.download(PRETRAINED_MODEL_URL)
        !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
else:
    print("Model Already Downloaded.")

## 6. Create Label Map pbtxt File
This cell creates the pbtxt file that your model will use to make predictions. The name of the file is set by the LABEL_MAP_NAME variable. <br>
The labels given here MUST MATCH the labels given to your image dataset. Otherwise an error will be generated when you try to run train your custom model.

**Format:** {'name':'CustomLabel', 'id':1}

Make sure to increase the id value by 1 for each label in your dataset. <br>
**Example:** {'name':'CustomLabel1', 'id':1}, {'name':'CustomLabel2', 'id':2}, {'name':'CustomLabel3', 'id':3}, {'name':'CustomLabel4', 'id':4}

In [ ]:
labels = [ {'name':'ThumbsUp', 'id':1}, {'name':'LiveLong', 'id':2} ]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

print("Label Map Created.")

## 7. Create TF records

In [ ]:
#Check if generate_tfrecord.py script has been downloaded
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
        
        
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

## 8. Copy Pre-Trained Model Config to Custom Model Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

## 9. Update Config For Transfer Learning

**Make sure to change the *pipeline_config.model.MODELTYPE.num_classes = len(labels)* to match the pre-trained model type specfied**

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
#Get data from current config file
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

""" 
Depending on the model type being used the line below will need to be changed.

If using Faster or Mask pre-trained model set line to
pipeline_config.model.faster_rcnn.num_classes = len(labels)

If using SSD or EfficientDet pre-trained model set line to
pipeline_config.model.ssd.num_classes = len(labels)

If using CenterNet pre-trained model set line to
pipeline_config.model.center_net.num_classes = len(labels)
""" 
pipeline_config.model.ssd.num_classes = len(labels)

#Sets location of training record, test record, and checkpoint files
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

#Create new config file
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## 10. Create Train Model Command
**Copy printed command and run in terminal window**

In [ ]:
#Set number of training steps - Should be 10000-20000 for a full run - Use 1000-2000 for a quick test
training_steps = 5000

TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], training_steps)

print(command)

## 11. Evaluate the Model
**Copy printed command and run in terminal window** <br>
**Press ctrl+c in terminal to exit Evaluation Script**

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

print(command)

### Run Tensorboard 
**Run cell below and go to the following link:** http://localhost:6006/  <br>
**Press ctrl+c in terminal to exit Evaluation Script**

In [ ]:
command = "tensorboard --logdir=" + os.path.join(paths['CHECKPOINT_PATH'], 'eval')
print(command)

## 12. Import Libraries Needed to Run Custom Model

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## 13. Load Train Model From Checkpoint

In [ ]:
#Set checkpoint file to load
checkpoint = 'ckpt-1'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], checkpoint)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 14. Set Objects to Detect & Image to Detect them on

In [ ]:
#Sets the pbtxt file to use - specified by LABEL_MAP_NAME variable 
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

#Sets image to do object prediction on 
IMAGE_PATH = ("C:\\Users\Richard\\Desktop\\thumbsup.cd56334e-ff92-11ec-9566-84144db13b80.jpg")

## 15. Run Object Detection for Given Image

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

## 16. Freezing the Model's Graph & Save for Later
**Copy printed command and run in terminal window**

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

print(command)